# 0. PREPARATION

In [75]:
# Import all necessary packages
# Package for downloading data from open topography website
# Both geoapis and geofabrics to read the boundaries and download the files from open topography API
import geoapis
import geoapis.lidar
import geofabrics
import glob                                                    # For getting all files' names in a certain path

# Package for reading las/laz file and handling lidar point cloud
import numpy as np                                             # For all calculation and data array/matrices manipulation
import pdal                                                    # For reading and writing las/laz file and handling lidar point cloud data

# Packages for transformation
from numba import njit, guvectorize, float64                   # For speeding up the time of the code running

# Packages for converting lidar point cloud las/laz file into DEM raster netcdf file
import os                                                      # For manipulating the directory path (create/change/move folders) and to execute commands
import pathlib                                                 # For manipulating the directory path
from pathlib import Path                                       # For creating folders
import shapely.geometry                                        # For creating a polygon object
import geopandas                                               # For creating a series to store the shapely geometry object
import shutil                                                  # For copying file/folder
import xarray                                                  # For creating and manipulating an array of raster
import rioxarray                                               # For manipulating spatial data under xarray array format (open and write .nc/.tiff files and set crs)
import json                                                    # For creating json text data format
from geofabrics import processor                               # For executing the command of converting lidar point cloud into a DEM raster

# Packages for unrotating and untranslating
from osgeo import gdal                                         # For manipulating rasters (calculating centers)
import rasterio.features                                       # For vectorising features in array
from shapely.geometry import shape                             # For manipulating spatial information (geometry) under GeoJSON format
from shapely.geometry import Polygon                           # For creating polygons to change raster values
import geopandas as gpd                                        # For manupulating shape files
from pyogrio import write_dataframe                            # For writing out shape file (twice faster than geopandas)

# Package for manipulating spatial data
import rasterio                                                # For reading and manipulating spatial data

# Package for running software from outside of Python (commandline or PowerShell)
import subprocess                                              # For mainly running BG_Flood model

# Package for timing steps
import time                                                    # For timing steps

In [76]:
# Remove all warnings
# Reference: https://numba.pydata.org/numba-doc/dev/reference/deprecation.html#suppressing-deprecation-warnings
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning, NumbaWarning
import warnings

warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaWarning)

In [77]:
# Assign the name you have just created for the drive
drive = "S"
main_folder = "MULTIPLE_TILES_Monte_Carlo_BGFLOOD"
version = "version_1"
header = f"{drive}:\\{main_folder}\\{version}"

# Create header path
pathlib.Path(f"{header}").mkdir(parents=True, exist_ok=True)

In [78]:
# Move into the folder where all the data are stored
# Reference: https://social.technet.microsoft.com/Forums/en-US/cef10e47-2fe1-482b-be62-ae74116a4e5f/mapped-network-drive-is-offline?forum=win10itpronetworking
import os
os.getcwd()
os.chdir(f"{header}")

In [79]:
# Create paths

#--------------------------------------------------- This is for TRANSFORMATION PART -------------------------------------------


# 0_open_topography: Folder stores ownloaded lidar point cloud data from open topography website
# Reference: https://portal.opentopography.org/datasets
original_lidar_path = f"{header}\\0_open_topography"

# 1_modified_lidar: Folder stores ownloaded lidar point cloud data from open topography website
modified_lidar_path = f"{header}\\1_modified_lidar"

# 2_transformation: Folder stores transformed lidar point cloud data
rotated_lidar_path = f"{header}\\2_transformation\\rotated_lidar"
translated_lidar_path = f"{header}\\2_transformation\\translated_lidar"
combined_lidar_path = f"{header}\\2_transformation\\combined_lidar"

# 3_dem_raster: Folder stores DEM rasters
# Rotation
# Netcdf
rotated_nc_raster_path = f"{header}\\3_dem_raster\\rotated_dem_raster\\rot_dem_raster\\rot_netcdf"
# GeoTiff
rotated_tiff_raster_path = f"{header}\\3_dem_raster\\rotated_dem_raster\\rot_dem_raster\\rot_tiff"
# Ascii
rotated_asc_raster_path = f"{header}\\3_dem_raster\\rotated_dem_raster\\rot_dem_raster\\rot_ascii"
rotated_elements_path = f"{header}\\3_dem_raster\\rotated_dem_raster\\rot_elements"

# Translation
# Netcdf
translated_nc_raster_path = f"{header}\\3_dem_raster\\translated_dem_raster\\tra_dem_raster\\tra_netcdf"
# GeoTiff
translated_tiff_raster_path = f"{header}\\3_dem_raster\\translated_dem_raster\\tra_dem_raster\\tra_tiff"
# Ascii
translated_asc_raster_path = f"{header}\\3_dem_raster\\translated_dem_raster\\tra_dem_raster\\tra_ascii"
translated_elements_path = f"{header}\\3_dem_raster\\translated_dem_raster\\tra_elements"

# Combination
# Netcdf
combined_nc_raster_path = f"{header}\\3_dem_raster\\combined_dem_raster\\com_dem_raster\\com_netcdf"
# GeoTiff
combined_tiff_raster_path = f"{header}\\3_dem_raster\\combined_dem_raster\\com_dem_raster\\com_tiff"
# Ascii
combined_asc_raster_path = f"{header}\\3_dem_raster\\combined_dem_raster\\com_dem_raster\\com_ascii"
combined_elements_path = f"{header}\\3_dem_raster\\combined_dem_raster\\com_elements"

# 4_BG_Flood: Folder stores BG_Flood outputs and Flowdepth outputs
# BG_Flood output
rotated_BGoutput_path = f"{header}\\4_BG_Flood\\BG_Flood_output\\rotation\\rotated_BGoutput"
translated_BGoutput_path = f"{header}\\4_BG_Flood\\BG_Flood_output\\translation\\translated_BGoutput"
combined_BGoutput_path = f"{header}\\4_BG_Flood\\BG_Flood_output\\combination\\combined_BGoutput"

# BG_Flood param
rotated_BGparam_path = f"{header}\\4_BG_Flood\\BG_Flood_output\\rotation\\rotated_BGparam"
translated_BGparam_path = f"{header}\\4_BG_Flood\\BG_Flood_output\\translation\\translated_BGparam"
combined_BGparam_path = f"{header}\\4_BG_Flood\\BG_Flood_output\\combination\\combined_BGparam"

# 5_flowdepth: Folder stores flowdepth extracted from BG-FLOOD output
rotated_flowdepth_path = f"{header}\\5_flowdepth\\rotated_flowdepth"
translated_flowdepth_path = f"{header}\\5_flowdepth\\translated_flowdepth"
combined_flowdepth_path = f"{header}\\5_flowdepth\\combined_flowdepth"

# 6_un_transformation: Folder stores un-transformed raster
# un_rotation
unrotated_crs_path = f"{header}\\6_un_transformation\\un_rotation\\unrotated_crs"
unrotated_path = f"{header}\\6_un_transformation\\un_rotation\\unrotated_shp"

# un_translation
untranslated_crs_path = f"{header}\\6_un_transformation\\un_translation\\untranslated_crs"
untranslated_path = f"{header}\\6_un_transformation\\un_translation\\untranslated_shp"

# un_combination
uncombined_crs_path = f"{header}\\6_un_transformation\\un_combination\\uncombined_crs"
uncombined_path = f"{header}\\6_un_transformation\\un_combination\\uncombined_shp"

#--------------------------------------------------- This is for ANALYSIS PART ------------------------------------------------

# 7_results: Folder stores mean and standard deviation
# Un_rotation
csv_rotation = f"{header}\\7_results\\un_rotation\\unrot_csv"
raster_rotation = f"{header}\\7_results\\un_rotation\\unrot_raster"
polygon_rotation = f"{header}\\7_results\\un_rotation\\unrot_polygon"
plot_rotation = f"{header}\\7_results\\un_rotation\\unrot_plot"

# Un_translation
csv_translation = f"{header}\\7_results\\un_translation\\untra_csv"
raster_translation = f"{header}\\7_results\\un_trfanslation\\untra_raster"
polygon_translation = f"{header}\\7_results\\un_translation\\untra_polygon"
plot_translation = f"{header}\\7_results\\un_translation\\untra_plot"

# Un_combination
csv_combination = f"{header}\\7_results\\un_combination\\uncom_csv"
raster_combination = f"{header}\\7_results\\un_combination\\uncom_raster"
polygon_combination = f"{header}\\7_results\\un_combination\\uncom_polygon"
plot_combination = f"{header}\\7_results\\un_combination\\uncom_plot"

In [80]:
# Create a list of all paths
neccessary_files = [
    # 0_open_topography
    original_lidar_path,
    
    # 1_modified_lidar
    modified_lidar_path,
    
    # 2_transformation
    rotated_lidar_path,
    translated_lidar_path,
    combined_lidar_path,
    
    # 3_dem_raster
    # Rotation
    rotated_nc_raster_path,
    rotated_tiff_raster_path,
    rotated_asc_raster_path,
    rotated_elements_path,
    
    # Translation
    translated_nc_raster_path,
    translated_tiff_raster_path,
    translated_asc_raster_path,
    translated_elements_path,
    
    # Combination
    combined_nc_raster_path,
    combined_tiff_raster_path,
    combined_asc_raster_path,
    combined_elements_path,
    
    # 4_BG_FLOOD
    rotated_BGoutput_path,
    translated_BGoutput_path,
    combined_BGoutput_path,
    
    # 5_flowdepth
    rotated_flowdepth_path,
    translated_flowdepth_path,
    combined_flowdepth_path,
    
    # 6_un_transformation
    # Unrotation
    unrotated_crs_path,
    unrotated_path,
    
    # Untranslation
    untranslated_crs_path,
    untranslated_path,
    
    # Uncombination
    uncombined_crs_path,
    uncombined_path,
    
    # 7_results
    # Unrotation
    csv_rotation,
    raster_rotation,
    polygon_rotation,
    plot_rotation,
    
    # Untranslation
    csv_translation,
    raster_translation,
    polygon_translation,
    plot_translation,
    
    # Uncombination
    csv_combination,
    raster_combination,
    polygon_combination,
    plot_combination
]

In [81]:
# Create folder
# Reference: https://stackoverflow.com/questions/273192/how-can-i-safely-create-a-nested-directory-in-python
for each_folder in neccessary_files:
    pathlib.Path(f"{each_folder}").mkdir(parents=True, exist_ok=True)

# 1. DOWNLOADING LIDAR POINTS

## 1.1. Downloading LiDAR data

In [82]:
# Create a function to download las/laz files from open topography
def download_lidar(boundary_coordinates_func, lidar_number):
    '''This function is to download las/laz files from Open Topography
    
    Arguments:
                boundary_coordinates_func:
                                        A list of xmin, ymin, xmax, ymax of rectangular boundary
                lidar_number:
                                        Ordinal number of lidar folder
    '''
    # Get crs
    h_crs = 2193
    v_crs = 7839
    
    # Get xmin, ymin, xmax, ymax of rectangular boundary
    x0_func = boundary_coordinates_func[0]
    y0_func = boundary_coordinates_func[1]
    x1_func = boundary_coordinates_func[2]
    y1_func = boundary_coordinates_func[3]
    
    # Assign those coordinates into shapely geometry polygon and store under geopandas format
    lidar_bound_coordinates = shapely.geometry.Polygon([(x0_func, y0_func), (x1_func, y0_func), (x1_func, y1_func), (x0_func, y1_func)])
    lidar_bound_coordinates = geopandas.GeoSeries([lidar_bound_coordinates])
    lidar_bound_coordinates = lidar_bound_coordinates.set_crs(h_crs)
    
    # Create lidar folder to store neccessary files
    lidar_folder_dir = pathlib.Path(os.getcwd()) / pathlib.Path(f"{original_lidar_path}\\lidar_{lidar_number}")
    if not os.path.exists(lidar_folder_dir):
        os.mkdir(lidar_folder_dir)
    
    # Create test file to store test_catchment.zip file (one of neccessary files for downloading las/laz files)
    test_dir = pathlib.Path(os.getcwd()) / pathlib.Path(f"{original_lidar_path}\\lidar_{lidar_number}\\test")
    if not os.path.exists(test_dir):
        os.mkdir(test_dir)
        
    # Create test_catchment.zip
    test_path = pathlib.Path(f"{original_lidar_path}\\lidar_{lidar_number}\\test\\test_catchment")
    lidar_bound_coordinates.to_file(test_path)
    shutil.make_archive(base_name=test_path, format='zip', root_dir=test_path)
    shutil.rmtree(test_path)
    lidar_bound_path = pathlib.Path(str(test_path) + ".zip")
    
    # Create polygon
    lidar_polygon = geopandas.read_file(lidar_bound_path)
    lidar_polygon.to_crs(h_crs)
    
    # Download lidar
    lidar_fetcher = geoapis.lidar.OpenTopography(cache_path = fr"{original_lidar_path}\\lidar_{lidar_number}",
                                                 search_polygon = lidar_polygon,
                                                 verbose = True)
    lidar_fetcher.run()

## 1.2. Modifying LiDAR data

In [83]:
def modified_lidar(lidar_number, method="statistical", parameter1=100, parameter2=40):
    '''This function is to modify the LiDAR data before DEM generation process. 
       In this process, the noise/outliers in LiDAR data will be removed by one of two selected methods and boundary range.
       Method 1: Statistical outlier methods requires two parameters - mean_k (parameter1) and multiplier (parameter2)
       Method 2: Radius method requires two parameters - radius (parameter1) and min_k (parameter2)
       Boundary range: There are still some noise/outliers that cannot be caught properly by the seleted method
                       and thus another boundary range should be developed to remove them. The range is set from -150 to 3000 (meters)
       Please refer the reference part for more information
       
    Reference: https://pdal.io/stages/filters.outlier.html
               https://pdal.io/pipeline.html
               https://pdal.io/apps/translate.html
               
    Arguments:
               lidar_number:
                                Ordinal number of lidar folder
               method:
                                Name of the method - "statistical" and "radius"
                                The default is "statistical"
               parameter1:
                                if "statistical", parameter1 is figure for "mean_k", else it is figure for "radius".
                                The default is 100 for "mean_k"
               parameter2:
                                if "statistical", parameter2 is figure for "multiplier", else it is figure for "min_k".
                                The default is 40 for "multiplier"
    
    '''
    # Create folder for storing modified LiDAR data
    pathlib.Path(fr"{modified_lidar_path}\\lidar_{lidar_number}\\Wellington_2013").mkdir(parents=True, exist_ok=True)
    
    # Get lists of tile paths and names
    # Tile paths
    list_tile_paths = glob.glob(f"{original_lidar_path}\\lidar_{lidar_number}\\Wellington_2013\*.laz")
    # Tile names
    length_list = len(list_tile_paths)
    list_tile_names = [os.path.basename(list_tile_paths[name]) for name in range(length_list)]
    
    # Check and remove noise/outliers
    for each_tile in range(length_list):
        # Start timing modifying
        start_modifying = time.time()
        
        # Set the path of input file
        input_file_func = list_tile_paths[each_tile]
        
        # Set the path of output file
        output_filename = list_tile_names[each_tile]
        output_file_func = fr"{modified_lidar_path}\\lidar_{lidar_number}\\Wellington_2013\\{output_filename}"
        
        # Develop method under dictionary format with string contents
        if method == "statistical":
            method_content = {
                "type": "filters.outlier",
                "method": "statistical",
                "mean_k": parameter1,
                "multiplier": parameter2
            }
        else:
            method_content = {
                "type": "filters.outlier",
                "method": "radius",
                "radius": parameter1,
                "min_k": parameter2
            }
        
        # Set instruction to remove noise/outliers under json format
        remove_outliers_json = [
            input_file_func,
            method_content,
            {
                "type": "filters.range",
                "limits": "Classification![7:7],Z[-100:1000]"
            },
            {
                "type": "writers.las",
                "filename": f"{output_file_func}",
                "compression": "laszip"
            }
        ]
        
        remove_outliers_instruction = {"pipeline": remove_outliers_json}
        
        # Execute the command
        pdal_remove_outliers = pdal.Pipeline(json.dumps(remove_outliers_instruction))
        pdal_remove_outliers.execute()
        
        # End timing modifying
        end_modifying = time.time()
        
        # Separator
        modifying_time = end_modifying - start_modifying
        print("* Tile", each_tile, "-", list_tile_names[each_tile], ":                      ",
              modifying_time/60)
        
    # Copy the tile index
    shutil.copy2(fr"{original_lidar_path}\\lidar_{lidar_number}\\Wellington_2013\\Wellington_2013_TileIndex.zip",
                 fr"{modified_lidar_path}\\lidar_{lidar_number}\\Wellington_2013\\Wellington_2013_TileIndex.zip")

# 2. REFERENCE DEM RASTER

## 2.1. Padding box coordinates

In [84]:
# Create a function to calculate the distance of two points
def distance_calculation(point_1, point_2):
    '''This function is to calculate the distance of two points
    Reference: https://orion.math.iastate.edu/dept/links/formulas/form2.pdf
    Arguments:
                point_1:
                        First point
                point_2:
                        Second point
    Returns:
                distance:
                        Euclidean distance between two points 
    '''
    # Calculating the euclidean distance between two points
    distance = np.sqrt(np.power((point_1[0] - point_2[0]),2) + np.power((point_1[1] - point_2[1]),2))
    return distance

In [85]:
def get_divisible_number(changed_number, divisible_number=16):
    '''This function is to find the padding number that is divisible by 16
    Reference: https://stackoverflow.com/questions/8002217/how-do-you-check-whether-a-number-is-divisible-by-another-number-python
    Arguments:
                changed_number:
                                Number that needs changing into the number that can be divisible by 16
                divisible_number:
                                Default is 16
    Return:
                changed_number:
                                A new number that can divisible by 16
    '''
    # Get the number that is divisible by 16
    while changed_number % divisible_number != 0:
        changed_number = changed_number + 1
        
    return changed_number

In [86]:
def padding_combination(coordinates_func, transformation_selection, addition):
    '''This function is to create the coordinates of padding/grid size
    Argument:
                coordinates_func:
                                        A list of xmin, ymin, xmax, ymax.
                transformation_selection:
                                        "r" means rotation
                                        "t" means translation
                                        "c" means combination
                addition:
                                        A number used to extend the padding
                            
    Returns:
                padding_func:
                                        A list of x min, x max, y min and y max
    '''
    # Set up the path for transformation_selection
    if transformation_selection == 'r':
        transformed_nc_raster_path = rotated_nc_raster_path
    elif transformation_selection == 't':
        transformed_nc_raster_path = translated_nc_raster_path
    else:
        transformed_nc_raster_path = combined_nc_raster_path
            
    x_min = coordinates_func[0]
    y_min = coordinates_func[1]
    x_max = coordinates_func[2]
    y_max = coordinates_func[3]

    center_x_func = ((x_min + x_max)/2)
    center_y_func = ((y_min + y_max)/2)
    center_padding_func = np.array([center_x_func, center_y_func])

    # Calculate the radius
    radius = distance_calculation(np.array([x_min, y_min]), center_padding_func) # Choose randomly a corner coordinates of rectangle point cloud to calculate the distance with center point
    
    # Get the coordinates of middle points of 4 sides of the rectangle point cloud
    middle_point_1 = np.array([x_min, center_y_func])
    middle_point_2 = np.array([center_x_func, y_max])
    middle_point_3 = np.array([x_max, center_y_func])
    middle_point_4 = np.array([center_x_func, y_min])
    
    # Calculate the distances between these points and the center point
    distance_center_1 = distance_calculation(middle_point_1, center_padding_func)
    distance_center_2 = distance_calculation(middle_point_2, center_padding_func)
    distance_center_3 = distance_calculation(middle_point_3, center_padding_func)
    distance_center_4 = distance_calculation(middle_point_4, center_padding_func)
    
    # Calculate the distances between the above middle points and the middle points of 4 sides of the padding
    distance_1 = radius - distance_center_1
    distance_2 = radius - distance_center_2
    distance_3 = radius - distance_center_3
    distance_4 = radius - distance_center_4
    
    # Convert into numbers which will be divisible by 16
    distance_padding_1 = get_divisible_number(np.ceil(distance_1))
    distance_padding_2 = get_divisible_number(np.ceil(distance_2))
    distance_padding_3 = get_divisible_number(np.ceil(distance_3))
    distance_padding_4 = get_divisible_number(np.ceil(distance_2))
    
    # Calculate the coordinates of the middle points of 4 sides of the padding
    padding_middle_point_1 = np.array((middle_point_1[0] - distance_padding_1, center_y_func))
    padding_middle_point_2 = np.array((center_x_func, middle_point_2[1] + distance_padding_2))
    padding_middle_point_3 = np.array((middle_point_3[0] + distance_padding_3, center_y_func))
    padding_middle_point_4 = np.array((center_x_func, middle_point_4[1] - distance_padding_4))
    
    # Calculate the coordinates of 4 corners of the padding
    padding_x_min = np.min(np.array([padding_middle_point_1[0],
                                     padding_middle_point_2[0],
                                     padding_middle_point_3[0],
                                     padding_middle_point_4[0]])) - addition

    padding_x_max = np.max(np.array([padding_middle_point_1[0],
                                     padding_middle_point_2[0],
                                     padding_middle_point_3[0],
                                     padding_middle_point_4[0]])) + addition

    padding_y_min = np.min(np.array([padding_middle_point_1[1],
                                     padding_middle_point_2[1],
                                     padding_middle_point_3[1],
                                     padding_middle_point_4[1]])) - addition

    padding_y_max = np.max(np.array([padding_middle_point_1[1],
                                     padding_middle_point_2[1],
                                     padding_middle_point_3[1],
                                     padding_middle_point_4[1]])) + addition
    padding_func = [padding_x_min, padding_y_min, padding_x_max, padding_y_max]
    
    return padding_func

## 2.2. Reference DEM raster

In [87]:
# Create a function to create a DEM raster from transformed lidar data
def dem_raster_reference(transformation_selection, filename_lidar_opentopo, padding_func, padding=True):
    ''' This function is to create a raster file from a las/laz file
    
    Arguments:
                transformation_selection:
                                            "r" means rotation
                                            "t" means translation
                                            "c" means combination
                filename_lidar_opentopo:
                                            Name of the folder storing LiDAR data downloaded from Opentopography
                padding_func:
                                            A list of x min, x max, y min and y max
                padding:
                                            if True, padding, else, no padding. Default is True
    '''
    # Set up the path for transformation_selection
    if transformation_selection == 'r':
        transformed_lidar_path = rotated_lidar_path
        nc_raster_path_func = rotated_nc_raster_path
        elements_path_func = rotated_elements_path
        transformed = "rotated"
    elif transformation_selection == 't':
        transformed_lidar_path = translated_lidar_path
        nc_raster_path_func = translated_nc_raster_path
        elements_path_func = translated_elements_path
        transformed = "translated"
    else:
        transformed_lidar_path = combined_lidar_path
        nc_raster_path_func = combined_nc_raster_path
        elements_path_func = combined_elements_path
        transformed = "combined"
    
    # Name of output
    if padding:
        output_name = "generated_dem_reference.nc"
        catchment_boundary = "catchment_boundary_reference"
    else:
        output_name = "generated_dem_no_padding.nc"
        catchment_boundary = "catchment_boundary_no_padding"
    
    # Set crs
    h_crs = 2193
    v_crs = 7839
    resolution = 10
    
    # Create folder for storing data materials
    data_dir = pathlib.Path(os.getcwd()) / pathlib.Path(f"{elements_path_func}\\data_{transformed}_ref")
    if not os.path.exists(data_dir):
        os.mkdir(data_dir)
    
    # Bounding box
    x0 = padding_func[0]; y0 = padding_func[1]; x1 = padding_func[2]; y1 = padding_func[3];
    catchment = shapely.geometry.Polygon([(x0, y0), (x1, y0), (x1, y1), (x0, y1)])
    catchment = geopandas.GeoSeries([catchment])
    catchment = catchment.set_crs(h_crs)

    catchment_path = pathlib.Path(f"{elements_path_func}\\data_{transformed}_ref\\{catchment_boundary}")
    catchment.to_file(catchment_path)
    shutil.make_archive(base_name=catchment_path, format='zip', root_dir=catchment_path)
    shutil.rmtree(catchment_path)

    
    # Design JSON instructions
    instructions = {"instructions":
                    {
                        "output": {
                            "crs": {
                                "horizontal": h_crs,
                                "vertical": v_crs
                            },
                            "grid_params": {
                                "resolution": resolution
                            }
                        },
                        "apis": {
                            "open_topography": {
                                "Wellington_2013": {
                                    "crs": {
                                        "horizontal": h_crs,
                                        "vertical": v_crs
                                    }
                                }
                            }
                        },
                        "processing": {
                            "chunk_size": 100,
                            "number_of_cores": 4
                        },
                        "data_paths": {
                            "local_cache": f"{modified_lidar_path}\\{filename_lidar_opentopo}",
                            "catchment_boundary": f"{elements_path_func}\\data_{transformed}_ref\\{catchment_boundary}.zip",
                            "land": f"{elements_path_func}\\data_{transformed}_ref\\{catchment_boundary}.zip",
                            "dense_dem_extents": f"{elements_path_func}\\data_{transformed}_ref\\extents.geojson",
                            "result_dem": f"{nc_raster_path_func}\\{output_name}"
                        },
                        "general": {
                            "set_dem_shoreline": True,
                            "drop_offshore_lidar": True,
                            "keep_only_ground_lidar": False,
                            "interpolation_missing_values": True
                        }
                    }
                   }
    
    
    # Create DEM raster
    runner = processor.DemGenerator(instructions)
    runner.run()

# 3. TRANSFORMATION PROCESS

## 3.1. Reading las/laz files of tiles

In [88]:
# Create a function to read las/laz file
def read_las_file(filename_tile):
    ''' This function is to read the las/las file into arrays
    Reference: https://pythonhosted.org/laspy/file.html
               https://pythonhosted.org/laspy/tut_part_1.html
    Arguments:
                filename_tile:
                                Name of las/laz files of tiles

    Returns:
                points_classification_func:
                                An array includes classification information (int format)
                coordinates_func:
                                An array includes x, y, z coordinates values of points as rows
    '''
    # Set up horizontal and vertical crs
    h_crs = 2193
    v_crs = 7839

    
    # Write las/las file into arrays
    pdal_pipeline_instructions = [{"type":"readers.las", "filename": fr"{filename_tile}"},
                                  {"type":"filters.reprojection","in_srs":f"EPSG:{h_crs}+{v_crs}",
                                   "out_srs":f"EPSG:{h_crs}+{v_crs}"}]

    pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
    pdal_pipeline.execute()

    # Call out arrays
    points_full_func = pdal_pipeline.arrays # note is shape [1,nx,ny]
    
    # Assign x, y , z (values in meters)
    x_func = points_full_func[0]["X"]
    y_func = points_full_func[0]["Y"]
    z_func = points_full_func[0]["Z"]
    
    # Get classification
    point_classification_func = points_full_func[0]['Classification']
    
    
    # Write x, y, z into arrays with each row as (x, y, z) (meters as unit)
    coordinates_func = np.vstack((x_func, y_func, z_func)).transpose().astype('float64')
    
    return point_classification_func, coordinates_func

## 3.2. Transformation method

### 3.2.1. Rotation

In [89]:
def center_calculation(transformation_selection, lidar=True):
    '''This function is to calculate the center of reference DEM raster which will be used in the whole process
    Reference: https://gis.stackexchange.com/questions/104362/how-to-get-extent-out-of-geotiff
               https://rasterio.readthedocs.io/en/latest/quickstart.html
    Arguments:
                transformation_selection:
                                        "r" means rotation
                                        "t" means translation
                                        "c" means combination
                lidar:
                                        True means doing rotation on LiDAR data
                                        False means doing unrotation on model outputs
    Return:
                A tuple of x, y coordinates of center raster
    '''
    # Set up the path for transformation_selection
    if transformation_selection == 'r':
        transformed_nc_raster_path = rotated_nc_raster_path
    elif transformation_selection == 't':
        transformed_nc_raster_path = translated_nc_raster_path
    else:
        transformed_nc_raster_path = combined_nc_raster_path
            
    raster_reference_func = gdal.Open(fr"{transformed_nc_raster_path}\\generated_dem_reference.nc")
    if lidar:
        xmin, xpixel, _, ymax, _, ypixel = raster_reference_func.GetGeoTransform()
        width, height = raster_reference_func.RasterXSize, raster_reference_func.RasterYSize
        xmax = xmin + width * xpixel
        ymin = ymax + height * ypixel
        center_x_func = ((xmin + xmax)/2)
        center_y_func = ((ymin + ymax)/2)
        center_func = np.array([center_x_func, center_y_func])
    else:
        center_func = ((raster_reference_func.RasterXSize)/2, (raster_reference_func.RasterYSize)/2)
    return center_func

In [90]:
# Create a command for guvectorize() decorator
# In that, float64[:,:] stands for the format of the array and float64 stands for the format of other parameters
# (m,n) signature stands for the matrix of the array and () signature stands for other parameters
gu_rotation = guvectorize([(float64[:,:], float64, float64, float64, float64, float64[:,:])], '(m,n),(),(),(),()->(m,n)')

# Create a point rotation function
def point_rotation(coordinates_func, angle, center_x_func, center_y_func, clockwise, new_coordinates_func):
    '''This function is to calculate the rotated coordinates of lidar data
    Reference: https://www.youtube.com/watch?v=RqZH-7hlI48
               https://stackoverflow.com/questions/14607640/rotating-a-vector-in-3d-space
               https://stackoverflow.com/questions/5954603/transposing-a-1d-numpy-array
               https://en.wikipedia.org/wiki/Rotation_matrix
               https://math.stackexchange.com/questions/270194/how-to-find-the-vertices-angle-after-rotation
               
               https://github.com/numba/numba/issues/3312
               https://numba.pydata.org/numba-doc/latest/user/vectorize.html
               https://numba.pydata.org/numba-doc/latest/cuda/ufunc.html
               http://numba.pydata.org/numba-doc/0.20.0/reference/compilation.html
               http://numba.pydata.org/numba-doc/0.12/tutorial_numpy_and_numba.html
               https://numba.pydata.org/numba-doc/dev/reference/types.html
    Arguments:
               coordinates_func:
                                An array of the coordinates of the point will be rotated
               angle: 
                                The value of angle to rotate
               center_x_func and center_y_func:
                                Values of the coordinates of center of the point cloud
               clockwise:
                                Rotating the points in clockwise (1) or anti_clockwise (0) directions
               new_coordinates_func:
                                A new array of rotated x, y, z coordinates values of a point
    '''
    # Convert degree to radian and calculate cosine and sine
    radian = np.deg2rad(angle)
    cosine = np.cos(radian)
    sine = np.sin(radian)
    
    # Create a for loop to manipulate each row of the array
    for i in range(coordinates_func.shape[0]):
        # Do substraction with center coordinates
        diff_x = coordinates_func[i, 0] - center_x_func
        diff_y = coordinates_func[i, 1] - center_y_func

        # Calculate the rotated point coordinates
        if clockwise == 0:                                                                  # Rotating in anti-clockwise direction
            new_coordinates_func[i, 0] = diff_x*cosine - diff_y*sine + center_x_func
            new_coordinates_func[i, 1] = diff_x*sine + diff_y*cosine + center_y_func
            new_coordinates_func[i, 2] = coordinates_func[i, 2]
        else:                                                                               # Rotating in clockwise direction
            new_coordinates_func[i, 0] = diff_x*cosine + diff_y*sine + center_x_func
            new_coordinates_func[i, 1] = diff_x*(-sine) + diff_y*cosine + center_y_func
            new_coordinates_func[i, 2] = coordinates_func[i, 2]

# Wrapping function to map later
wrapping_point_rotation = gu_rotation(point_rotation)

### 3.2.2. Translation

In [91]:
# Create a command for guvectorize() decorator
# In that, float64[:,:] stands for the format of the array and float64 stands for the format of other parameters
# (m,n) signature stands for the matrix of the array and () signature stands for other parameters
gu_translation = guvectorize([(float64[:,:], float64, float64, float64[:,:])], '(m,n),(),()->(m,n)')

# Create a point translation function
def point_translation(coordinates_func, x_translation_func, y_translation_func, new_coordinates_func):
    '''This function is to calculate the translated coordinates of lidar data
    Arguments:
                coordinates_func:
                                An array of the coordinates of the point will be rotated
                x_translation and y_translation:
                                Distances to translate x and y coordinates values
                new_coordinates_func:
                                A new array of translated x, y, z coordinates values of a point
    '''
    for i in range(coordinates_func.shape[0]):
        new_coordinates_func[i, 0] = coordinates_func[i, 0] + x_translation_func
        new_coordinates_func[i, 1] = coordinates_func[i, 1] + y_translation_func
        new_coordinates_func[i, 2] = coordinates_func[i, 2]

# Wrapping function to map later
wrapping_point_translation = gu_translation(point_translation)

## 3.3. Writing transformed las/laz files

In [114]:
# Create a function to write transformed las/laz file
def writing_las_file(transformation_selection, point_classification_func, number_simulation, transformed_array, filename_code):
    '''This function is to create a new lidar las/laz file the same as the old lidar las/laz file with different coordinates
    Arguments:
                transformation_selection:
                                            "r" means rotation
                                            "t" means translation
                                            "c" means combination
                point_classification_func:
                                            An array contains points' classification
                number_simulation:
                                            Ordinal number_simulation of simulation
                transformed_array:
                                            A transformed array
                filename_code:
                                            Code of file name (the same as original lidar tiles' names)
    '''
    # Set up horizontal and vertical crs
    h_crs = 2193
    v_crs = 7839
    
    # Output file
    if transformation_selection == 'r':
        pathlib.Path(fr"{rotated_lidar_path}\\rotated_lidar_{number_simulation}\\Wellington_2013").mkdir(parents=True, exist_ok=True)
        outfile = fr"{rotated_lidar_path}\\rotated_lidar_{number_simulation}\\Wellington_2013\\{filename_code}"
    elif transformation_selection == 't':
        pathlib.Path(fr"{translated_lidar_path}\\translated_lidar_{number_simulation}\\Wellington_2013").mkdir(parents=True, exist_ok=True)
        outfile = fr"{translated_lidar_path}\\translated_lidar_{number_simulation}\\Wellington_2013\\{filename_code}"
    else:
        pathlib.Path(fr"{combined_lidar_path}\\combined_lidar_{number_simulation}\\Wellington_2013").mkdir(parents=True, exist_ok=True)
        outfile = fr"{combined_lidar_path}\\combined_lidar_{number_simulation}\\Wellington_2013\\{filename_code}"
    
    # Get the shape of transformed LiDAR array
    lidar_arr_shape = len(point_classification_func)
    
    # Create empty transformed LiDAR array
    lidar_arr = np.empty([lidar_arr_shape], dtype=[('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Classification', 'u1')])
    
    # Overwrite the X, Y, and Z values in the points array with the rotated values
    lidar_arr['X'] = (transformed_array[:, 0]).flatten()
    lidar_arr['Y'] = (transformed_array[:, 1]).flatten()
    lidar_arr['Z'] = (transformed_array[:, 2]).flatten()
    lidar_arr['Classification'] = point_classification_func
    
    # Save the file
    pdal_pipeline_instructions = [
        {"type":  "writers.las",
         "scale_x":"auto",
         "scale_y":"auto",
         "offset_x":"auto",
         "offset_y":"auto",
         "a_srs": f"EPSG:{h_crs}+{v_crs}",
         "filename": str(outfile),
         "compression": "laszip"}
    ]
    
    # Run
    pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions), [lidar_arr])
    pdal_pipeline.execute()

## 3.4. Writing tile index file

In [93]:
def get_url(lidar_number, filename_tile):
    '''This function is to extract the URL of each tile from original tile index file
    
    Arguments:
                lidar_number:
                                Ordinal number of lidar folder
                filename_tile:
                                Name of each tile
    
    Return:
                url_file:
                                URL of the specific tile
                                
    '''
    # Read the shape file to have geopandas dataframe
    full_file = gpd.read_file(fr"{modified_lidar_path}\\lidar_{lidar_number}\\Wellington_2013\\Wellington_2013_TileIndex.zip")
    
    # Get the row with the file name the same as given 'filename_tile'
    file_info = full_file[full_file['Filename'] == filename_tile]
    
    # Get the index of that row
    url_index = file_info.index[0]
    
    # Use the extracted index to get the correct URL
    url_file = full_file['URL'][url_index]
    
    return url_file

In [94]:
def tile_index_polygon(transformation_selection, number_simulation, filename_tile):
    '''This function is to create transformed tile index polygon for each original tile
    
    Arguments:
                transformation_selection:
                                        "r" means rotation
                                        "t" means translation
                                        "c" means combination
                number_simulation:
                                        Ordinal number of simulation
                filename_tile:
                                        Name of each tile
    '''
    # Set up the path for transformation_selection
    if transformation_selection == 'r':
        transformed_lidar_path = rotated_lidar_path
        transformed = "rotated"
    elif transformation_selection == 't':
        transformed_lidar_path = translated_lidar_path
        transformed = "translated"
    else:
        transformed_lidar_path = combined_lidar_path
        transformed = "combined"
    
    # Get information of each tile las/laz file
    tile_info_func = subprocess.run(['pdal', 'info',
                                      fr"{transformed_lidar_path}\\{transformed}_lidar_{number_simulation}\\Wellington_2013\\{filename_tile}"],
                                    stdout = subprocess.PIPE)
    
    # Read by json
    json_tile_func = json.loads(tile_info_func.stdout.decode('utf-8'))
    
    # Extract boundary coordinates
    tile_boundary_func = np.array(json_tile_func['stats']['bbox']['native']['boundary']['coordinates'][0]).astype('float64')
    
    # Drop z column
    tile_boundary_adjusted_func = np.delete(tile_boundary_func, [2], 1)
    
    # Convert to polygon
    tile_boundary_polygon_func = Polygon(tile_boundary_adjusted_func)
    
    return tile_boundary_polygon_func

## 3.5. Transformation on tiles

In [95]:
def transformed_tiles_generation(transformation_selection, lidar_number,
                                 angle_func, x_translation_func, y_translation_func,
                                 center_x_func, center_y_func,
                                 number_simulation):
    '''This function is to create transformed tiles
    
    Arguments:
                transformation_selection:
                                                "r" means rotation
                                                "t" means translation
                                                "c" means combination
                lidar_number:
                                                Ordinal number of lidar folder
                angle_func, x_translation_func, y_translation_func:
                                                Angle, x, and y coordinates that are used to transform LiDAR data
                center_x_func, center_y_func:
                                                x and y coordinates of center point used to transform (particularly rotate)
                number_simulation:
                                                Ordinal number of simulation
                                            
    '''
    # Set up the path for transformation_selection
    if transformation_selection == 'r':
        transformed_lidar_path = rotated_lidar_path
        transformed = "rotated"
    elif transformation_selection == 't':
        transformed_lidar_path = translated_lidar_path
        transformed = "translated"
    else:
        transformed_lidar_path = combined_lidar_path
        transformed = "combined"
        
    # Get list of tiles names
    list_tile_paths = glob.glob(f"{modified_lidar_path}\\lidar_{lidar_number}\\Wellington_2013\*.laz")
    length_list = len(list_tile_paths)
    list_tile_names = [os.path.basename(list_tile_paths[name]) for name in range(length_list)]
    
    # Geometry list of tile index
    tile_index_list = []
    tile_url_list = []
    
    # Transform each tile
    for each_tile in range(length_list):

        # Start timing the whole transformation process ----------------------------------------------------------------
        start_transformation_process = time.time()
        
        # Reading tiles
        points_tiles, coordinates_tiles = read_las_file(list_tile_paths[each_tile])
        
        # Transform tiles
        rotated_array_func = wrapping_point_rotation(coordinates_tiles, angle_func, center_x_func, center_y_func, 0)
        combined_array_func = wrapping_point_translation(rotated_array_func, x_translation_func, y_translation_func)
        
        # Write LiDAR tiles into las/laz file
        writing_las_file(transformation_selection, points_tiles, 
                         number_simulation, combined_array_func, list_tile_names[each_tile])
        
        
        # Write tile index polygon
        tile_polygon = tile_index_polygon(transformation_selection, number_simulation, list_tile_names[each_tile])
        tile_index_list.append(tile_polygon)
        
        # Write tile url polygon
        tile_url = get_url(lidar_number, list_tile_names[each_tile])
        tile_url_list.append(tile_url)
        
        # End timing the whole transformation process -----------------------------------------------------------------
        end_transformation_process = time.time()
        
        
        
        # PRINT -------------------------------------------------------------------------------------------------------
        # Print the running time of whole transformation process
        writing_transformation_process_time = end_transformation_process - start_transformation_process
        
        print("*", "Tile", each_tile, "-", list_tile_names[each_tile], ":                         ",
              writing_transformation_process_time/60)
        
        # End printing ------------------------------------------------------------------------------------------------
        
    print("-"*20)
    
    # Create geopandas dataframe containing tile index information
    tile_info = {
        "Filename": list_tile_names,
        "URL": tile_url_list
                }
    tile_data_index = gpd.GeoDataFrame(data = tile_info,
                                       geometry = tile_index_list,
                                       crs = 2193)
    
    # Create tiles shapefile
    tile_index_path = pathlib.Path(fr"{transformed_lidar_path}\\{transformed}_lidar_{number_simulation}\\Wellington_2013\\Wellington_2013_TileIndex")
    tile_data_index.to_file(tile_index_path)
    shutil.make_archive(base_name=tile_index_path, format='zip', root_dir=tile_index_path)
    shutil.rmtree(tile_index_path)

# 4. DERIVING DEM RASTER FROM LIDAR DATA

## 4.1. Deriving DEM raster from LiDAR data

In [96]:
# Create a function to create a DEM raster from transformed lidar data
def dem_raster(transformation_selection, number, padding_func):
    ''' This function is to create a raster file from a las/laz file
    
    Arguments:
                transformation_selection:
                            "r" means rotation
                            "t" means translation
                            "c" means combination
                transformed_las_file:
                            A path to the transformed las/laz file
                padding_func:
                            A list of x min, x max, y min and y max
    '''
    # Set up the path for transformation_selection
    if transformation_selection == 'r':
        transformed_lidar_path = rotated_lidar_path
        nc_raster_path_func = rotated_nc_raster_path
        elements_path_func = rotated_elements_path
        transformed = "rotated"
    elif transformation_selection == 't':
        transformed_lidar_path = translated_lidar_path
        nc_raster_path_func = translated_nc_raster_path
        elements_path_func = translated_elements_path
        transformed = "translated"
    else:
        transformed_lidar_path = combined_lidar_path
        nc_raster_path_func = combined_nc_raster_path
        elements_path_func = combined_elements_path
        transformed = "combined"
    
    
    # Set crs
    h_crs = 2193
    v_crs = 7839
    resolution = 10
    
    # Create folder for storing data materials
    data_dir = pathlib.Path(os.getcwd()) / pathlib.Path(f"{elements_path_func}\\data_{transformed}_{number}")
    if not os.path.exists(data_dir):
        os.mkdir(data_dir)
    
    # Bounding box
    x0 = padding_func[0]; y0 = padding_func[1]; x1 = padding_func[2]; y1 = padding_func[3];
    catchment = shapely.geometry.Polygon([(x0, y0), (x1, y0), (x1, y1), (x0, y1)])
    catchment = geopandas.GeoSeries([catchment])
    catchment = catchment.set_crs(h_crs)

    catchment_path = pathlib.Path(f"{elements_path_func}\\data_{transformed}_{number}\\catchment_boundary")
    catchment.to_file(catchment_path)
    shutil.make_archive(base_name=catchment_path, format='zip', root_dir=catchment_path)
    shutil.rmtree(catchment_path)
    
    
    # Design JSON instructions
    instructions = {"instructions":
                    {
                        "output": {
                            "crs": {
                                "horizontal": h_crs,
                                "vertical": v_crs
                            },
                            "grid_params": {
                                "resolution": resolution
                            }
                        },
                        "apis": {
                            "open_topography": {
                                "Wellington_2013": {
                                    "crs": {
                                        "horizontal": h_crs,
                                        "vertical": v_crs
                                    }
                                }
                            }
                        },
                        "processing": {
                            "chunk_size": 100,
                            "number_of_cores": 4
                        },
                        "data_paths": {
                            "local_cache": f"{transformed_lidar_path}\\{transformed}_lidar_{number}",
                            "catchment_boundary": f"{elements_path_func}\\data_{transformed}_{number}\\catchment_boundary.zip",
                            "land": f"{elements_path_func}\\data_{transformed}_{number}\\catchment_boundary.zip",
                            "dense_dem_extents": f"{elements_path_func}\\data_{transformed}_{number}\\extents.geojson",
                            "result_dem": f"{nc_raster_path_func}\\generated_dem_{transformed}_{number}.nc"
                        },
                        "general": {
                            "set_dem_shoreline": True,
                            "drop_offshore_lidar": True,
                            "keep_only_ground_lidar": False,
                            "interpolation_missing_values": True
                        }
                    }
                   }
    
    # Create DEM raster
    runner = processor.DemGenerator(instructions)
    runner.run()

## 4.2. Creating polygon boundaries

In [97]:
# Create a function to create a polygon layer to change values
def polygon_generation(polygon_boundary, transformation_selection, number_simulation,
                       angle_func, x_translation_func, y_translation_func,
                       center_x_func, center_y_func,
                       filename_output_polygon):
    '''This function is to create polygon layer to change the pixel values
    
    Arguments:
                polygon_boundary:
                                        A list of coordinates of 
                transformation_selection:
                                        "r" means rotation
                                        "t" means translation
                                        "c" means combination
                number_simulation:
                                        Ordinal number of simulation
                angle_func, x_translation_func, y_translation_func:
                                                Angle, x, and y coordinates that are used to transform LiDAR data
                center_x_func, center_y_func:
                                                x and y coordinates of center point used to transform (particularly rotate)
                filename_output_polygon:
                                        Output file name
    '''
    # Set up the path for transformation_selection
    if transformation_selection == 'r':
        nc_raster_path_func = rotated_nc_raster_path
        tiff_raster_path_func = rotated_tiff_raster_path
        transformed = "rotated"
    elif transformation_selection == 't':
        nc_raster_path_func = translated_nc_raster_path
        tiff_raster_path_func = translated_tiff_raster_path
        transformed = "translated"
    else:
        nc_raster_path_func = combined_nc_raster_path
        tiff_raster_path_func = combined_tiff_raster_path
        transformed = "combined"
        
    # Create storing folder to store the file
    polygon_dir = pathlib.Path(os.getcwd()) / pathlib.Path(f"{tiff_raster_path_func}\\{transformed}_{number_simulation}")
    if not os.path.exists(polygon_dir):
        os.mkdir(polygon_dir)
        
    # Convert the list into array to do transformation
    boundary_coordinates = np.array(polygon_boundary).astype('float64')
    
    # Transform
    rotated_boundary = wrapping_point_rotation(boundary_coordinates, angle_func, center_x_func, center_y_func, 0)
    translated_boundary = wrapping_point_translation(rotated_boundary, x_translation_func, y_translation_func)
    transformed_boundary = translated_boundary
    
    # Convert into shapely geometry Polygon
    shapely_polygon = Polygon(transformed_boundary)
    
    # Create geopandas dataframe
    polygon_dataframe = gpd.GeoDataFrame(geometry = [shapely_polygon],
                                         crs = 2193)
    
    # Write into shape file
    polygon_dataframe.to_file(fr"{filename_output_polygon}.shp")

In [98]:
# Create a function to create polygon boundaries by using polygon_generation() function
def polygon_boundaries(transformation_selection, number_simulation,
                       angle_func, x_translation_func, y_translation_func,
                       center_x_func, center_y_func):
    '''This function is to create polygon boundaries by using polygon_generation() function.
       These polygons will be used to change the pixel values inside or outside.
       
    Reference: https://gis.stackexchange.com/questions/414194/changing-raster-pixel-values-outside-of-polygon-box-using-rioxarray
    
    Arguments:
                transformation_selection:
                                        "r" means rotation
                                        "t" means translation
                                        "c" means combination
                number_simulation:
                                        Ordinal number of simulation
                angle_func, x_translation_func, y_translation_func:
                                                Angle, x, and y coordinates that are used to transform LiDAR data
                center_x_func, center_y_func:
                                                x and y coordinates of center point used to transform (particularly rotate)
                file_name:
                                        Output file name
    '''
    # Set up the path for transformation_selection
    if transformation_selection == 'r':
        nc_raster_path_func = rotated_nc_raster_path
        tiff_raster_path_func = rotated_tiff_raster_path
        transformed = "rotated"
    elif transformation_selection == 't':
        nc_raster_path_func = translated_nc_raster_path
        tiff_raster_path_func = translated_tiff_raster_path
        transformed = "translated"
    else:
        nc_raster_path_func = combined_nc_raster_path
        tiff_raster_path_func = combined_tiff_raster_path
        transformed = "combined"
        
    # Define the path for output
    output_path = fr"{tiff_raster_path_func}\\{transformed}_{number_simulation}"
        
    # PADDING #####################################################
    # Get extent information of original no-padding raster
    raster_no_padding = rasterio.open(fr"{nc_raster_path_func}\\generated_dem_no_padding.nc")
    no_padding_xmin, no_padding_ymin, no_padding_xmax, no_padding_ymax = raster_no_padding.bounds
    
    # Create polygon to remove PADDING values
    # Set up polygon coordinates into a list
    polygon_padding = [(no_padding_xmax, no_padding_ymax),
                       (no_padding_xmax, no_padding_ymin),
                       (no_padding_xmin, no_padding_ymin),
                       (no_padding_xmin, no_padding_ymax),
                       (no_padding_xmax, no_padding_ymax)]
    
    # Create polygon padding layer under shape file
    polygon_generation(polygon_padding, 
                       transformation_selection,
                       number_simulation,
                       angle_func, x_translation_func, y_translation_func,
                       center_x_func, center_y_func,
                       f"{output_path}\\polygon_padding_{number_simulation}")
    
    
#     # BOUNDARY 1 ###################################################
#     # Get extent information for first boundary
#     boundary_1_ymin = no_padding_ymin + 10*8
#     boundary_1_xmax = no_padding_xmax
#     boundary_1_ymax = boundary_1_ymin + 10*10
#     boundary_1_xmin = no_padding_xmax - 10
    
#     # Write into a list
#     polygon_boundary_1 = [(boundary_1_xmax, boundary_1_ymax),
#                           (boundary_1_xmax, boundary_1_ymin),
#                           (boundary_1_xmin, boundary_1_ymin),
#                           (boundary_1_xmin, boundary_1_ymax),
#                           (boundary_1_xmax, boundary_1_ymax)]
    
#     # Create polygon boundary 1 under shape file
#     polygon_generation(polygon_boundary_1, 
#                        transformation_selection,
#                        number_simulation,
#                        angle_func, x_translation_func, y_translation_func,
#                        center_x_func, center_y_func,
#                        f"{output_path}\\polygon_boundary_1_{number_simulation}")
    
    
    
#     # BOUNDARY 2 ###################################################
#     # Get extent information from raster including padding
#     raster_padding = rasterio.open(fr"{nc_raster_path_func}\\generated_dem_reference.nc")
#     padding_xmin, padding_ymin, padding_xmax, padding_ymax = raster_padding.bounds
    
#     # Get coordinates of boundary 2
#     boundary_2_ymax = no_padding_ymin
#     boundary_2_xmax = padding_xmax
#     boundary_2_ymin = padding_ymin
#     boundary_2_xmin = padding_xmin
    
#     # Write into a list
#     polygon_boundary_2 = [(boundary_2_xmax, boundary_2_ymax),
#                           (boundary_2_xmax, boundary_2_ymin),
#                           (boundary_2_xmin, boundary_2_ymin),
#                           (boundary_2_xmin, boundary_2_ymax),
#                           (boundary_2_xmax, boundary_2_ymax)]
    
    
#     # Create polygon boundary 2 under shape file
#     polygon_generation(polygon_boundary_2, 
#                        transformation_selection,
#                        number_simulation,
#                        angle_func, x_translation_func, y_translation_func,
#                        center_x_func, center_y_func,
#                        f"{output_path}\\polygon_boundary_2_{number_simulation}")

## 4.3. Changing values based on polygons

In [99]:
# Create a function to changing values inside or outside polygons
def value_change(shapefile_func, file_need_changing_func, value_func, inside=True):
    '''This function is to change pixel values inside or outside polygons
    
    Reference: https://corteva.github.io/rioxarray/html/rioxarray.html
               https://corteva.github.io/rioxarray/stable/examples/convert_to_raster.html
               https://gis.stackexchange.com/questions/414194/changing-raster-pixel-values-outside-of-polygon-box-using-rioxarray
               https://automating-gis-processes.github.io/CSC/notebooks/L2/geopandas-basics.html
               https://corteva.github.io/rioxarray/stable/getting_started/nodata_management.html
               https://gdal.org/programs/gdal_translate.html
               https://gis.stackexchange.com/questions/390438/replacing-nodata-values-by-a-constant-using-gdal-cli-tools
               
    Arguments:
                shapefile_func:
                                Polygon boundaries
                file_need_changing:
                                File contains values that need changing
                value_func:
                                Values used to replace
                inside:
                                If True, change values inside, else, change values outside
    '''
    if inside:
        inside_command = fr"gdal_rasterize -burn {value_func} {shapefile_func} {file_need_changing_func}"
        os.system(inside_command)
    else:
        outside_command = fr"gdal_rasterize -i -burn {value_func} {shapefile_func} {file_need_changing_func}"
        os.system(outside_command)

In [100]:
# Create a function to execute the value changes
def value_change_execution(transformation_selection, number_simulation):
    ''' This function is to apply value_change() function on a certain raster
    
    Arguments:
                transformation_selection:
                                        "r" means rotation
                                        "t" means translation
                                        "c" means combination
                number_simulation:
                                        Ordinal number of simulation
    '''
    # Set up the path for transformation_selection
    if transformation_selection == 'r':
        nc_raster_path_func = rotated_nc_raster_path
        tiff_raster_path_func = rotated_tiff_raster_path
        transformed = "rotated"
    elif transformation_selection == 't':
        nc_raster_path_func = translated_nc_raster_path
        tiff_raster_path_func = translated_tiff_raster_path
        transformed = "translated"
    else:
        nc_raster_path_func = combined_nc_raster_path
        tiff_raster_path_func = combined_tiff_raster_path
        transformed = "combined"
    
    # Convert NetCDF file into GeoTiff file
    raster_nc = rioxarray.open_rasterio(fr"{nc_raster_path_func}\\generated_dem_{transformed}_{number_simulation}.nc")
    raster_nc.rio.to_raster(fr"{tiff_raster_path_func}\\{transformed}_{number_simulation}\\generated_dem_{transformed}_{number_simulation}.tif")
    
    # Get polygon shape_file and file that has values need changing
    file_path = fr"{tiff_raster_path_func}\\{transformed}_{number_simulation}"
    shapefile_padding_dir = fr"{file_path}\\polygon_padding_{number_simulation}.shp"
    shapefile_boundary_1_dir = fr"{file_path}\\polygon_boundary_1_{number_simulation}.shp"
    shapefile_boundary_2_dir = fr"{file_path}\\polygon_boundary_2_{number_simulation}.shp"
    file_need_changing_dir = fr"{file_path}\\generated_dem_{transformed}_{number_simulation}.tif"
    
    ## Changing the value
    # Changing padding values into -999
    value_change(shapefile_padding_dir, file_need_changing_dir, 999, False)
    
#     # Changing boundary 1 values into 100
#     value_change(shapefile_boundary_1_dir, file_need_changing_dir, 100, True)
    
#     # Changing boundary 2 values into 100
#     value_change(shapefile_boundary_2_dir, file_need_changing_dir, 100, True)

## 4.4. Convert file into ASCII file

In [101]:
# Create a function to convert GeoTiff file into ASCII file
def convert_to_asc(transformation_selection, number_simulation):
    '''This function is to convert GeoTiff file into ASCII file
    
    Arguments:
                transformation_selection:
                                        "r" means rotation
                                        "t" means translation
                                        "c" means combination
                number_simulation:
                                        Ordinal number of simulation
    '''
    # Set up the path for transformation_selection
    if transformation_selection == 'r':
        tiff_raster_path_func = rotated_tiff_raster_path
        asc_raster_path_func = rotated_asc_raster_path
        transformed = "rotated"
    elif transformation_selection == 't':
        tiff_raster_path_func = translated_tiff_raster_path
        asc_raster_path_func = translated_asc_raster_path
        transformed = "translated"
    else:
        tiff_raster_path_func = combined_tiff_raster_path
        asc_raster_path_func = combined_asc_raster_path
        transformed = "combined"
        
    # Convert GeoTiff file into ASCII file
    tiff_raster_func = rioxarray.open_rasterio(fr"{tiff_raster_path_func}\\{transformed}_{number_simulation}\\generated_dem_{transformed}_{number_simulation}.tif")
    
#     nodata_tiff_raster_func = tiff_raster_func.rio.write_nodata(-999, inplace=True)
#     crs_tiff_raster_func = nodata_tiff_raster_func.rio.write_crs(2193)
    
    crs_tiff_raster_func = tiff_raster_func.rio.write_crs(2193)
    crs_tiff_raster_func.rio.to_raster(fr"{asc_raster_path_func}\\generated_dem_{transformed}_{number_simulation}.asc")

# 5. BG-FLOOD

In [102]:
# Create a function to write river and BG_Flood parameters into text files
def river_and_param(transformation_selection, number_simulation, river_box_transformation):
    ''' This function is to create two text files including river text file and BG param text file
    
    Reference: https://www.pythontutorial.net/python-basics/python-write-text-file/
    
    Arguments:
                transformation_selection:
                                         "r" means rotation
                                         "t" means translation
                                         "c" means combination
                number_simulation:
                                         Ordinal number of simulation
                river_box_transformation:
                                         An array of a transformed x and y array
    '''
    # River information
    river = ['0.0,150.0',
             '300.0,300.0',
             '1800.0,300.0']
    
    river = [
        '0.0,10.0',
        '100.0,15.0',
        '200.0,10.0',
        '300.0,20.0',
        '400.0,25.0',
        '500.0,30.0',
        '600.0,50.0',
        '700.0,75.0',
        '800.0,80.0',
        '900.0,100.0',
        '1000.0,150.0',
        '1200.0,40.0',
        '1400.0,50.0',
        '1800.0,55.0',
        '2000.0,100.0',
        '2500.0,150.0',
        '2600.0,80.0',
        '2700.0,75.0',
        '2800.0,50.0',
        '3000.0,25.0',
        '3600.0,20.0',
        '3700.0,10.0',
        '3800.0,15.0',
        '3900.0,10.0',
        '4000.0,20.0',
        '4100.0,25.0',
        '4200.0,30.0',
        '4300.0,50.0',
        '4500.0,75.0',
        '4700.0,80.0',
        '4800.0,35.0',
        '5000.0,45.0',
        '5100.0,40.0',
        '5200.0,50.0',
        '5500.0,55.0',
        '5800.0,100.0',
        '6000.0,250.0',
        '6200.0,80.0',
        '6500.0,75.0',
        '6800.0,50.0',
        '7000.0,25.0',
        '7200.0,20.0'
    ]
    
    
    
    river_param_path = r'P:\Courses\PhD\BG_flood2\BG-Flood_Win10_v0.6-a'

    # Set up the path for transformation_selection
    if transformation_selection == "r":
        trans = 'rot'
        dem_raster_path_func = f"{rotated_asc_raster_path}\\generated_dem_rotated_{number_simulation}.asc"
        output_path = fr"{rotated_BGoutput_path}\\Output_rotated_{number_simulation}.nc"
        param_path = rotated_BGparam_path
    elif transformation_selection == "t":
        trans = 'tra'
        dem_raster_path_func = f"{translated_asc_raster_path}\\generated_dem_translated_{number_simulation}.asc"
        output_path = fr"{translated_BGoutput_path}\\Output_translated_{number_simulation}.nc"
        param_path = translated_BGparam_path
    else:
        trans = 'com'
        dem_raster_path_func = f"{combined_asc_raster_path}\\generated_dem_combined_{number_simulation}.asc"
        output_path = fr"{combined_BGoutput_path}\\Output_combined_{number_simulation}.nc"
        param_path = combined_BGparam_path
        
    # Check values of xmin and xmax
    if river_box_transformation[0, 0] < river_box_transformation[1, 0]:
        xmin_river = river_box_transformation[0, 0]
        xmax_river = river_box_transformation[1, 0]
    else:
        xmin_river = river_box_transformation[1, 0]
        xmax_river = river_box_transformation[0, 0]
        
    # Check values of ymin and ymax
    if river_box_transformation[0, 1] < river_box_transformation[1, 1]:
        ymin_river = river_box_transformation[0, 1]
        ymax_river = river_box_transformation[1, 1]
    else:
        ymin_river = river_box_transformation[1, 1]
        ymax_river = river_box_transformation[0, 1]
        
    # BG parameters
    BG_param = [f'bathy = {dem_raster_path_func}?idw;',
                'gpudevice = 0;',
                'mask = 1500.0;',
                'dx = 10;',
                'smallnc = 0;',
                'outputtimestep = 100.0;',
                'endtime = 7200.0;',
                f'river = RiverDis.txt, {xmin_river}, {xmax_river}, {ymin_river}, {ymax_river};',
                f'outfile = {output_path};']
        
    # Write river text file
    with open(f'{river_param_path}\\RiverDis.txt', 'w') as riv:
        riv.write('\n'.join(river))
        
    # Write param text file
    with open(f'{river_param_path}\\BG_param.txt', 'w') as param:
        param.write('\n'.join(BG_param))
    
    # Copy the text files for later checking
    # Create the file
    pathlib.Path(f'{param_path}\\param_{number_simulation}').mkdir(parents=True, exist_ok=True)
    # River
    shutil.copy2(f'{river_param_path}\\BG_param.txt', f'{param_path}\\param_{number_simulation}\\RiverDis.txt')
    # BG param
    shutil.copy2(f'{river_param_path}\\BG_param.txt', f'{param_path}\\param_{number_simulation}\\BG_param.txt')

In [103]:
def run_BG_Flood(transformation_selection, number_simulation, river_box_transformation):
    '''This function is to run BG_Flood
    
    Arguments:
                transformation_selection:
                                         "r" means rotation
                                         "t" means translation
                                         "c" means combination
                number_simulation:
                                         Ordinal number of simulation
                river_box_transformation:
                                         An array of a transformed x and y array
    '''    
    # Set up the path for transformation_selection
    if transformation_selection == "r":
        river_and_param(transformation_selection, number_simulation, river_box_transformation)
        os.chdir(r'P:\\Courses\\PhD\\BG_flood2\\BG-Flood_Win10_v0.6-a')
        subprocess.check_output([".\\BG_Flood_Cleanup.exe"])
    elif transformation_selection == "t":
        river_and_param(transformation_selection, number_simulation, river_box_transformation)
        os.chdir(r'P:\\Courses\\PhD\\BG_flood2\\BG-Flood_Win10_v0.6-a')
        subprocess.check_output([".\\BG_Flood_Cleanup.exe"])
    else:
        river_and_param(transformation_selection, number_simulation, river_box_transformation)
        os.chdir(r'P:\\Courses\\PhD\\BG_flood2\\BG-Flood_Win10_v0.6-a')
        pathlib.Path('BG_log.txt').unlink(missing_ok=True)
        subprocess.check_output([".\BG_Flood_Cleanup.exe"])
    
    # Change the path back to origin
    os.chdir(f"{drive}:\\{main_folder}\\{version}")

# 6. EXTRACTING FLOWDEPTH

In [104]:
# Create function to extract the flowdepth from BG_Flood output
def flowdepth_extraction(transformation_selection, number, flowdepth_rate_func, time):
    '''This function is to extract the flowdepth from BG_Flood output
    Arguments:
                transformation_selection:
                            "r" means rotation
                            "t" means translation
                            "c" means combination
                number:
                            Ordinal number of simulation
                
                flowdepth_rate_func:
                            Threshold for flowdepth
                
                time:
                            Amount of time that BG_flood model predicted
    '''
    # Setting up the path for transformation_selection
    if transformation_selection == 'r':
        BGoutput = rotated_BGoutput_path
        flowdepth_path_func = rotated_flowdepth_path
        transformed = "rotated"
    elif transformation_selection == 't':
        BGoutput = translated_BGoutput_path
        flowdepth_path_func = translated_flowdepth_path
        transformed = "translated"
    else:
        BGoutput = combined_BGoutput_path
        flowdepth_path_func = combined_flowdepth_path
        transformed = "combined"
    

    output = f'Output_{transformed}_{number}.nc'
    
    command = f'gmt grdmath {BGoutput}\\{output}?h_P0[{time}] DUP {flowdepth_rate_func} GT MUL = {flowdepth_path_func}\\flowdepth_{transformed}_{number}_at_{time}.nc'
    print(command)
    os.system(command)

# 7. UN-TRANSFORMATION

In [105]:
def add_crs(transformation_selection, number, time):
    '''This function is to add crs into flowdepth extraction raster
    
    Arguments:  transformation_selection:
                        "r" means rotation
                        "t" means translation
                        "c" means combination
                angle:
                        Angle to unrotate
                time:
                        Amount of time that BG_flood model predicted
    '''
    # Set up the path for transformation_selection
    if transformation_selection == 'r':
        transformed = "rotated"
        flowdepth_path = rotated_flowdepth_path
        crs_path = unrotated_crs_path
        transformed_path = unrotated_path
    elif transformation_selection == 't':
        transformed = "translated"
        flowdepth_path = translated_flowdepth_path
        crs_path = untranslated_crs_path
        transformed_path = untranslated_path
    else:
        transformed = "combined"
        flowdepth_path = combined_flowdepth_path
        crs_path = uncombined_crs_path
        transformed_path = uncombined_path
    
    
    # Add crs to ROTATED raster (flowdepth files extracted from model outputs)
    raster_add_crs = xarray.open_dataset(fr"{flowdepth_path}\\flowdepth_{transformed}_{number}_at_{time}.nc")
    raster_add_crs = raster_add_crs.rio.write_crs(2193)
    raster_add_crs.rio.to_raster(fr"{crs_path}\\flowdepth_{transformed}_{number}_at_{time}.nc")

In [106]:
def polygons_uncombination(transformation_selection, number_simulation, angle_func, x_translation_func, y_translation_func, time):
    '''This function is to unrotate and write rasters into shapefiles
    
    Reference: https://docs.dea.ga.gov.au/notebooks/Frequently_used_code/Polygonise_pixel_edges.html
               https://gis.stackexchange.com/questions/187877/how-to-polygonize-raster-to-shapely-polygons/
               
               https://github.com/sgillies/affine/blob/master/affine/__init__.py#L178
               https://gis.stackexchange.com/questions/408386/rotating-raster-using-python
               https://pythonhosted.org/PyAgg/affine.m.html
               
               https://gis.stackexchange.com/questions/408386/rotating-raster-using-python
               https://gis.stackexchange.com/questions/350526/gdal-setgeotransform-issue
               https://corteva.github.io/rioxarray/stable/examples/convert_to_raster.html
               https://gdal.org/tutorials/geotransforms_tut.html#geotransforms-tut
               https://gdal.org/tutorials/raster_api_tut.html
               
    Arguments:  transformation_selection:
                                                "r" means rotation
                                                "t" means translation
                                                "c" means combination
                number_simulation:
                        Ordinal number of simulation
                angle_func, x_translation_func, y_translation_func:
                                                Angle, x, and y coordinates that are used to transform LiDAR data
                center_x_func, center_y_func:
                                                x and y coordinates of center point used to transform (particularly rotate)
                time:
                                                Amount of time that BG_flood model predicted
    '''
    # Set up the path for transformation_selection
    # Set up the path for transformation_selection
    if transformation_selection == 'r':
        transformed = "rotated"
        flowdepth_path = rotated_flowdepth_path
        crs_path = unrotated_crs_path
        untransformed_path = unrotated_path
    elif transformation_selection == 't':
        transformed = "translated"
        flowdepth_path = translated_flowdepth_path
        crs_path = untranslated_crs_path
        untransformed_path = untranslated_path
    else:
        transformed = "combined"
        flowdepth_path = combined_flowdepth_path
        crs_path = uncombined_crs_path
        untransformed_path = uncombined_path
    
    # Extract the array from COMBINED raster
    raster_poly = rasterio.open(fr"{crs_path}\\flowdepth_{transformed}_{number_simulation}_at_{time}.nc")
    raster_array = raster_poly.read(1)
    raster_transform = raster_poly.transform
    raster_crs = raster_poly.crs
    
    # Extract parameters: id and depth
    id_pixels = np.arange(raster_array.size).reshape(raster_array.shape)
    depth_list = list(raster_array.flatten())
    
    
    # UNCOMBINE 1 (untranslate) affine transformation
    uncombined1_transform = raster_transform * raster_transform.translation(x_translation_func, y_translation_func)
    
    # Find out centers
    raster_center = center_calculation('c', False)
    
    # UNCOMBINE 2 (unrotate) affine transformation
    uncombined2_transform = uncombined1_transform * uncombined1_transform.rotation(angle_func, raster_center)
    
    # Change the name
    uncombined_transform = uncombined2_transform
    
    # Vectorise features
    uncombined_vectors = rasterio.features.shapes(source = id_pixels.astype(np.int16),
                                                    transform = uncombined_transform)
    
    # List the UNCOMBINED polygons to extract neccessary parameters
    uncombined_vectors_list = list(uncombined_vectors)
    
    # Get geometry
    polygons_geometry_values = [shape(polygon_geometry) for polygon_geometry, value_geometry in uncombined_vectors_list]
    
    # Get id
    id_uncombined_pixels_values = [id_value for id_polygon, id_value in uncombined_vectors_list]
    
    # Create UNCOMBINED database under geopandas dataframe (gdf) format
    uncombined_data = {"id": id_uncombined_pixels_values,
                      "depth": depth_list}
    uncombined_raster_poly_gdf = gpd.GeoDataFrame(data = uncombined_data,
                                                  geometry = polygons_geometry_values,
                                                  crs = raster_crs)
    shapefile_dir = pathlib.Path(os.getcwd()) / pathlib.Path(f"{untransformed_path}\\shapefile_{number_simulation}")
    if not os.path.exists(shapefile_dir):
        os.mkdir(shapefile_dir)
    
    # Write geopandas dataframe to shape file
    write_dataframe(uncombined_raster_poly_gdf, fr"{untransformed_path}\\shapefile_{number_simulation}\\flowdepth_un{transformed}_{number_simulation}_at_{time}.shp", 
                    driver='ESRI Shapefile')

-------------------------------------------------------------------------------------------------------------------------------

# EXECUTING

In [107]:
%%time
# GET NECCESSARY INFORMATION

# LiDAR number file
num_lidar_file = 0

# Construct a list of boundary
resolution = 10
number_pixel_x = 16*9                                   # multiple of 16
number_pixel_y = 16*6                                   # multiple of 16

xmin = 1771385
ymin = 5472254
xmax = xmin + resolution * number_pixel_x
ymax = ymin + resolution * number_pixel_y

# Boundary for the area of interest
boundary_1 = [xmin, ymin, xmax, ymax]

# Padding boundary for DEMs (use this boundary to create padding)
addition_2 = 16*5
boundary_2 = padding_combination(boundary_1, "c", addition_2)

# Boundary for tiles (use this boundary to download LiDAR)
addition_3 = 16*5
boundary_3 = padding_combination(boundary_2, "c", addition_3)

# Extracting flowdepth rate (for flowdepth_extraction() function)
flowdepth_rate = 0

# Time to extract flowdepth (for flowdepth_extraction() function)
time_extract = 72

# Rivers
river_xy_coordinates = np.array([[1772824, 5472314.684], [1772825, 5472315.684]]).astype('float64')

# Set up the angle for rotation
angle_start = 0
angle_end = 90
angle_interval = 45

# Set up the translation
value_start = 0
value_end = 0
value_interval = 1

Wall time: 997 µs


In [108]:
# Check boundaries
print(boundary_1)
print(boundary_2)
print(boundary_3)

[1771385, 5472254, 1772825, 5473214]
[1771145.0, 5471774.0, 1773065.0, 5473694.0]
[1770665.0, 5471294.0, 1773545.0, 5474174.0]


In [109]:
%%time
# Download LiDAR
download_lidar(boundary_3, num_lidar_file)

Check files in dataset Wellington_2013
checking size: Wellington_2013\ot_CL1_WLG_2013_1km_089038.laz: 22160061, total (GB): 0.020638165064156055
checking size: Wellington_2013\ot_CL1_WLG_2013_1km_087038.laz: 25236171, total (GB): 0.04414118081331253
checking size: Wellington_2013\ot_CL1_WLG_2013_1km_089040.laz: 25801639, total (GB): 0.06817082967609167
checking size: Wellington_2013\ot_CL1_WLG_2013_1km_088038.laz: 25857820, total (GB): 0.09225280117243528
checking size: Wellington_2013\ot_CL1_WLG_2013_1km_086039.laz: 25949694, total (GB): 0.11642033699899912
checking size: Wellington_2013\ot_CL1_WLG_2013_1km_088039.laz: 25792909, total (GB): 0.14044185541570187
checking size: Wellington_2013\ot_CL1_WLG_2013_1km_086040.laz: 22502212, total (GB): 0.16139867343008518
checking size: Wellington_2013\ot_CL1_WLG_2013_1km_086041.laz: 25329480, total (GB): 0.18498858995735645
checking size: Wellington_2013\ot_CL1_WLG_2013_1km_089041.laz: 24493969, total (GB): 0.20780037622898817
checking size: 

In [110]:
%%time
# Modify LiDAR
modified_lidar(num_lidar_file, "statistical", 100, 40)

* Tile 0 - ot_CL1_WLG_2013_1km_086038.laz :                       2.3156893094380697
* Tile 1 - ot_CL1_WLG_2013_1km_086039.laz :                       2.270197856426239
* Tile 2 - ot_CL1_WLG_2013_1km_086040.laz :                       2.234510107835134
* Tile 3 - ot_CL1_WLG_2013_1km_086041.laz :                       2.403064012527466
* Tile 4 - ot_CL1_WLG_2013_1km_087038.laz :                       2.2254531542460123
* Tile 5 - ot_CL1_WLG_2013_1km_087039.laz :                       2.1897761106491087
* Tile 6 - ot_CL1_WLG_2013_1km_087040.laz :                       2.1427695115407306
* Tile 7 - ot_CL1_WLG_2013_1km_087041.laz :                       2.2150489886601767
* Tile 8 - ot_CL1_WLG_2013_1km_088038.laz :                       2.1979863047599792
* Tile 9 - ot_CL1_WLG_2013_1km_088039.laz :                       2.1867376883824665
* Tile 10 - ot_CL1_WLG_2013_1km_088040.laz :                       2.3433727741241457
* Tile 11 - ot_CL1_WLG_2013_1km_088041.laz :                       

In [111]:
%%time
# Generating reference DEMs
# Reference DEM without padding
dem_raster_reference("c", f"lidar_{num_lidar_file}", boundary_1, False)

# Reference DEM with padding
dem_raster_reference("c", f"lidar_{num_lidar_file}", boundary_2)

Check files in dataset Wellington_2013


C:\Program Files\Anaconda3\envs\geofabrics39\lib\site-packages\distributed\node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 63833 instead
  warnings.warn(


<Client: 'tcp://127.0.0.1:63834' processes=4 threads=4, memory=31.69 GiB>
Check files in dataset Wellington_2013


C:\Program Files\Anaconda3\envs\geofabrics39\lib\site-packages\distributed\node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 63939 instead
  warnings.warn(


<Client: 'tcp://127.0.0.1:63940' processes=4 threads=4, memory=31.69 GiB>
Wall time: 1min 20s


In [112]:
# Calculate coordinates of center point
center_point = center_calculation('c', True)
center_x = center_point[0]                     # Extract x coordinate of center point
center_y = center_point[1]                     # Extract y coordinate of center point

In [115]:
for angle_value1 in range(angle_start, angle_end+1, angle_interval):
    for value_x1 in range(value_start, value_end+1, value_interval):
        for value_y1 in range(value_start, value_end+1, value_interval):
            # Set up translation values for x and y coordinates.
            # Apply translation values into lidar point cloud array (under meters unit)
            x_translation_lidar = value_x1
            y_translation_lidar = value_y1
            
            # Set the name of each transformation
            combination_number1 = f"angle_{angle_value1}_x_{value_x1}_y_{value_y1}"
            
            # Print the header of the first running
            print("{0} angle: {1} and x: {2} and y: {3} {0}".format("-"*30, angle_value1, value_x1, value_y1))
            
            
            # Start timing tranforming process -------------------------------------------------------------
            start_transform_tiles = time.time()
            # Transform tiles
            transformed_tiles_generation(
                'c', num_lidar_file,
                angle_value1, value_x1, value_y1,
                center_x, center_y,
                combination_number1
            )
            # End timing generating DEM process
            end_transform_tiles = time.time()
            
            
            # Start timing generating DEM process -----------------------------------------------------------
            start_dem = time.time()
            # Generate DEM
            dem_raster("c", combination_number1, boundary_2)
            # Start timing generating DEM process
            end_dem = time.time()
            
            
            # PRINT -----------------------------------------------------------------------------------------
            # Print the transforming tiles process 
            transform_tile_time = end_transform_tiles - start_transform_tiles
            print("* Running time of transforming tiles:                               ",
                  transform_tile_time/60)
            
            # Print the running time of generating DEM process
            dem_time = end_dem - start_dem
            print("* Running time of generating DEM process:                           ",
                  dem_time/60)
            
            
            # Print separator
            print("-"*90)
            print("\n")

------------------------------ angle: 0 and x: 0 and y: 0 ------------------------------
* Tile 0 - ot_CL1_WLG_2013_1km_086038.laz :                          0.3093505342801412
* Tile 1 - ot_CL1_WLG_2013_1km_086039.laz :                          0.2939431548118591
* Tile 2 - ot_CL1_WLG_2013_1km_086040.laz :                          0.24449899991353352
* Tile 3 - ot_CL1_WLG_2013_1km_086041.laz :                          0.25724332332611083
* Tile 4 - ot_CL1_WLG_2013_1km_087038.laz :                          0.2570426066716512
* Tile 5 - ot_CL1_WLG_2013_1km_087039.laz :                          0.2644611239433289
* Tile 6 - ot_CL1_WLG_2013_1km_087040.laz :                          0.25711582899093627
* Tile 7 - ot_CL1_WLG_2013_1km_087041.laz :                          0.26151430209477744
* Tile 8 - ot_CL1_WLG_2013_1km_088038.laz :                          0.2562392036120097
* Tile 9 - ot_CL1_WLG_2013_1km_088039.laz :                          0.2546016653378805
* Tile 10 - ot_CL1_WLG_2013

C:\Program Files\Anaconda3\envs\geofabrics39\lib\site-packages\distributed\node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 49346 instead
  warnings.warn(


<Client: 'tcp://127.0.0.1:49347' processes=4 threads=4, memory=31.69 GiB>
* Running time of transforming tiles:                                4.161394476890564
* Running time of generating DEM process:                            0.7056979417800904
------------------------------------------------------------------------------------------


------------------------------ angle: 45 and x: 0 and y: 0 ------------------------------
* Tile 0 - ot_CL1_WLG_2013_1km_086038.laz :                          0.2710619568824768
* Tile 1 - ot_CL1_WLG_2013_1km_086039.laz :                          0.26555747191111245
* Tile 2 - ot_CL1_WLG_2013_1km_086040.laz :                          0.24273114999135334
* Tile 3 - ot_CL1_WLG_2013_1km_086041.laz :                          0.25582668781280515
* Tile 4 - ot_CL1_WLG_2013_1km_087038.laz :                          0.2594075520833333
* Tile 5 - ot_CL1_WLG_2013_1km_087039.laz :                          0.2575612187385559
* Tile 6 - ot_CL1_WLG_2013_1km_087040

C:\Program Files\Anaconda3\envs\geofabrics39\lib\site-packages\distributed\node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 49860 instead
  warnings.warn(


<Client: 'tcp://127.0.0.1:49861' processes=4 threads=4, memory=31.69 GiB>
* Running time of transforming tiles:                                4.0651137669881185
* Running time of generating DEM process:                            0.7703202684720357
------------------------------------------------------------------------------------------


------------------------------ angle: 90 and x: 0 and y: 0 ------------------------------
* Tile 0 - ot_CL1_WLG_2013_1km_086038.laz :                          0.26850149234135945
* Tile 1 - ot_CL1_WLG_2013_1km_086039.laz :                          0.26456689834594727
* Tile 2 - ot_CL1_WLG_2013_1km_086040.laz :                          0.23741631507873534
* Tile 3 - ot_CL1_WLG_2013_1km_086041.laz :                          0.2581955035527547
* Tile 4 - ot_CL1_WLG_2013_1km_087038.laz :                          0.2553750912348429
* Tile 5 - ot_CL1_WLG_2013_1km_087039.laz :                          0.25590330759684243
* Tile 6 - ot_CL1_WLG_2013_1km_0870

C:\Program Files\Anaconda3\envs\geofabrics39\lib\site-packages\distributed\node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 50281 instead
  warnings.warn(


<Client: 'tcp://127.0.0.1:50282' processes=4 threads=4, memory=31.69 GiB>
* Running time of transforming tiles:                                4.054940251509349
* Running time of generating DEM process:                            0.6244746247927347
------------------------------------------------------------------------------------------




In [116]:
%%time
# Execute changing values code
for angle_value2 in range(angle_start, angle_end+1, angle_interval):
    for value_x2 in range(value_start, value_end+1, value_interval):
        for value_y2 in range(value_start, value_end+1, value_interval):
            # Set up translation values for x and y coordinates.
            # Apply translation values into lidar point cloud array (under meters unit)
            x_translation_lidar = value_x2
            y_translation_lidar = value_y2
            
            # Set the name of each translation
            combination_number2 = f"angle_{angle_value2}_x_{value_x2}_y_{value_y2}"
            
            # Print the header of the first running
            print("{0} angle: {1} and x: {2} and y: {3} {0}".format("-"*30, angle_value2, value_x2, value_y2))
            
            # Start timing the whole process
            start_changing_values = time.time()

            # Create polygon boundaries
            polygon_boundaries("c", combination_number2,
                               angle_value2, x_translation_lidar, y_translation_lidar,
                               center_x, center_y)

            # Changing pixels values
            value_change_execution("c", combination_number2)

            # Convert to ASCII
            convert_to_asc("c", combination_number2)

            # End timing the whole process
            end_changing_values = time.time()

            # Print the running time of the whole process
            changing_values_time = end_changing_values - start_changing_values
            print("* Running time of the whole exchanging process:                 ",
                  changing_values_time/60)

            # Print separator
            print("-"*90)
            print("\n")

------------------------------ angle: 0 and x: 0 and y: 0 ------------------------------


* Running time of the whole exchanging process:                  0.01874312957127889
------------------------------------------------------------------------------------------


------------------------------ angle: 45 and x: 0 and y: 0 ------------------------------


* Running time of the whole exchanging process:                  0.014667308330535889
------------------------------------------------------------------------------------------


------------------------------ angle: 90 and x: 0 and y: 0 ------------------------------


* Running time of the whole exchanging process:                  0.014353116353352865
------------------------------------------------------------------------------------------


Wall time: 2.87 s


In [117]:
# Execute BG_Flood model
for angle_value3 in range(angle_start, angle_end+1, angle_interval):
    for value_x3 in range(value_start, value_end+1, value_interval):
        for value_y3 in range(value_start, value_end+1, value_interval):
            # Set up translation values for x and y coordinates.
            # Apply translation values into lidar point cloud array (under meters unit)
            x_translation_lidar = value_x3
            y_translation_lidar = value_y3

            # Set the name of each translation
            combination_number3 = f"angle_{angle_value3}_x_{value_x3}_y_{value_y3}"

            # Print the header of the first running
            print("{0} angle: {1} and x: {2} and y: {3} {0}".format("-"*30, angle_value3, value_x3, value_y3))

            # Start timing the whole model running process
            start_model = time.time()

            # River box coordinates
            rotated_river_xy_coordinates = wrapping_point_rotation(river_xy_coordinates, angle_value3, center_x, center_y, 0)
            combined_river_xy_coordinates = wrapping_point_translation(rotated_river_xy_coordinates, x_translation_lidar, y_translation_lidar)
            
            # Run BG_Flood
            run_BG_Flood("c", combination_number3, combined_river_xy_coordinates)

            # End timing the whole model running process
            end_model = time.time()

            # Print the running time of running BG_Flood model
            model_time = end_model - start_model
            print("* Running time of running BG_Flood model:", 
                  model_time/60)

            # Print separator
            print("-"*90)
            print("\n")

------------------------------ angle: 0 and x: 0 and y: 0 ------------------------------
* Running time of running BG_Flood model: 10.311006593704224
------------------------------------------------------------------------------------------


------------------------------ angle: 45 and x: 0 and y: 0 ------------------------------
* Running time of running BG_Flood model: 11.742301805814106
------------------------------------------------------------------------------------------


------------------------------ angle: 90 and x: 0 and y: 0 ------------------------------
* Running time of running BG_Flood model: 10.154708206653595
------------------------------------------------------------------------------------------




In [118]:
# Execute flowdepth extraction
for angle_value4 in range(angle_start, angle_end+1, angle_interval):
    for value_x4 in range(value_start, value_end+1, value_interval):
        for value_y4 in range(value_start, value_end+1, value_interval):
            # Set the name of each translation
            combination_number4 = f"angle_{angle_value4}_x_{value_x4}_y_{value_y4}"

            # Print the header of the first running
            print("{0} angle: {1} and x: {2} and y: {3} {0}".format("-"*30, angle_value4, value_x4, value_y4))
    
            # Start timing the whole extracting process
            start_extracting = time.time()

            # Extract flowdepth
            flowdepth_extraction("c", combination_number4, flowdepth_rate, time_extract)

            # End timing the whole extracting process
            end_extracting = time.time()

            # Print the running time of the extracting process
            extracting_time = end_extracting - start_extracting
            print("* Running time of extracting flowdepth:", 
                  extracting_time/60)

            # Print separator
            print("-"*90)
            print("\n")

------------------------------ angle: 0 and x: 0 and y: 0 ------------------------------
gmt grdmath S:\MULTIPLE_TILES_Monte_Carlo_BGFLOOD\version_1\4_BG_Flood\BG_Flood_output\combination\combined_BGoutput\Output_combined_angle_0_x_0_y_0.nc?h_P0[72] DUP 0 GT MUL = S:\MULTIPLE_TILES_Monte_Carlo_BGFLOOD\version_1\5_flowdepth\combined_flowdepth\flowdepth_combined_angle_0_x_0_y_0_at_72.nc
* Running time of extracting flowdepth: 0.026252373059590658
------------------------------------------------------------------------------------------


------------------------------ angle: 45 and x: 0 and y: 0 ------------------------------
gmt grdmath S:\MULTIPLE_TILES_Monte_Carlo_BGFLOOD\version_1\4_BG_Flood\BG_Flood_output\combination\combined_BGoutput\Output_combined_angle_45_x_0_y_0.nc?h_P0[72] DUP 0 GT MUL = S:\MULTIPLE_TILES_Monte_Carlo_BGFLOOD\version_1\5_flowdepth\combined_flowdepth\flowdepth_combined_angle_45_x_0_y_0_at_72.nc
* Running time of extracting flowdepth: 0.004171542326609294
------

In [119]:
# Execute adding crs function
for angle_value5 in range(angle_start, angle_end+1, angle_interval):
    for value_x5 in range(value_start, value_end+1, value_interval):
        for value_y5 in range(value_start, value_end+1, value_interval):
            # Set the name of each translation
            combination_number5 = f"angle_{angle_value5}_x_{value_x5}_y_{value_y5}"

            # Print the header of the first running
            print("{0} angle: {1} and x: {2} and y: {3} {0}".format("-"*30, angle_value5, value_x5, value_y5))
    
            # Start timing the whole adding crs process
            start_crs = time.time()

            # Add crs
            add_crs("c", combination_number5, time_extract)

            # End timing the whole adding crs process
            end_crs = time.time()

            # Print the running time of the adding crs process
            crs_time = end_crs - start_crs
            print("* Running time of extracting flowdepth:", 
                  crs_time/60)

            # Print separator
            print("-"*90)
            print("\n")

------------------------------ angle: 0 and x: 0 and y: 0 ------------------------------


* Running time of extracting flowdepth: 0.004469041029612223
------------------------------------------------------------------------------------------


------------------------------ angle: 45 and x: 0 and y: 0 ------------------------------
* Running time of extracting flowdepth: 0.00346837838490804
------------------------------------------------------------------------------------------


------------------------------ angle: 90 and x: 0 and y: 0 ------------------------------


* Running time of extracting flowdepth: 0.0035988291104634604
------------------------------------------------------------------------------------------




In [120]:
# Execute un-rotation
for angle_value6 in range(angle_start, angle_end+1, angle_interval):
    for value_x6 in range(value_start, value_end+1, value_interval):
        for value_y6 in range(value_start, value_end+1, value_interval):
            # Set up translation values for x and y coordinates.
            # Apply translation values into lidar point cloud array (under meters unit)
            x_translation_raster = value_x6 / 10 * (-1)
            y_translation_raster = value_y6 / 10

            # Set the name of each translation
            combination_number6 = f"angle_{angle_value6}_x_{value_x6}_y_{value_y6}"

            # Print the header of the first running
            print("{0} angle: {1} and x: {2} and y: {3} {0}".format("-"*30, angle_value6, value_x6, value_y6))

            # Start timing the whole un-combination process
            start_uncombination = time.time()

            # Un-combination
            polygons_uncombination('c', combination_number6, angle_value6, x_translation_raster, y_translation_raster, time_extract)

            # End timing the whole un-rotation process
            end_uncombination = time.time()            
            
            # Print the running time of the un-rotation process
            unrotation_time = end_uncombination - start_uncombination
            print("* Running time of unrotation:", 
                  unrotation_time/60)

            # Print separator
            print("-"*90)
            print("\n")

------------------------------ angle: 0 and x: 0 and y: 0 ------------------------------


* Running time of unrotation: 0.040973126888275146
------------------------------------------------------------------------------------------


------------------------------ angle: 45 and x: 0 and y: 0 ------------------------------


* Running time of unrotation: 0.03582177956899007
------------------------------------------------------------------------------------------


------------------------------ angle: 90 and x: 0 and y: 0 ------------------------------
* Running time of unrotation: 0.036017970244089766
------------------------------------------------------------------------------------------


